In [29]:
import pandas as pd
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import statsmodels.api as sm
import statsmodels.formula.api as smf
import lightgbm as lgb

In [30]:
#train
filename="/content/train.csv"
data = pd.read_csv(filename, encoding= 'unicode_escape',   parse_dates=[0,1,2,3])

In [31]:
l=preprocessing.LabelEncoder()
data['Holiday']=l.fit_transform(data['Holiday'])
data['Functioning Day']=l.fit_transform(data['Functioning Day'])
Date=pd.DataFrame({"year": data['Date'].dt.year,"month": data['Date'].dt.month,"day": data['Date'].dt.day})
data['year']=Date["year"]
data['month']=Date["month"]
data['day']=Date["day"]
data=data.drop(['Date'],axis=1)

In [32]:
data.describe()

,Humidity(%),Wind speed (m/s),Visibility (10m),Dew point temperature(Â°C),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm),Holiday,Functioning Day,year,month,day
count,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000,6000.00000,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000
mean,57.142667,1.816033,1362.965000,2.017383,0.578085,0.140050,0.08910,0.952000,0.992000,2017.876000,5.584000,14.912000
std,20.977885,1.067520,621.492068,14.003660,0.886947,1.074779,0.43548,0.213784,0.089092,0.329609,3.392471,9.332029
min,0.000000,0.000000,27.000000,-30.600000,0.000000,0.000000,0.00000,0.000000,0.000000,2017.000000,1.000000,1.000000
25%,41.000000,1.000000,820.000000,-9.300000,0.000000,0.000000,0.00000,1.000000,1.000000,2018.000000,3.000000,6.000000
50%,56.000000,1.600000,1546.000000,2.300000,0.020000,0.000000,0.00000,1.000000,1.000000,2018.000000,5.000000,15.000000
75%,73.000000,2.500000,1976.250000,14.000000,0.930000,0.000000,0.00000,1.000000,1.000000,2018.000000,7.000000,23.000000
max,98.000000,7.400000,2000.000000,26.800000,3.520000,35.000000,5.10000,1.000000,1.000000,2018.000000,12.000000,31.000000


In [33]:
data.shape

(6000, 16)

In [34]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6000 entries, 0 to 5999
Data columns (total 16 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Rented Bike Count           6000 non-null   object 
 1   Hour                        6000 non-null   object 
 2   Temperature(Â°C)            6000 non-null   object 
 3   Humidity(%)                 6000 non-null   int64  
 4   Wind speed (m/s)            6000 non-null   float64
 5   Visibility (10m)            6000 non-null   int64  
 6   Dew point temperature(Â°C)  6000 non-null   float64
 7   Solar Radiation (MJ/m2)     6000 non-null   float64
 8   Rainfall(mm)                6000 non-null   float64
 9   Snowfall (cm)               6000 non-null   float64
 10  Seasons                     6000 non-null   object 
 11  Holiday                     6000 non-null   int64  
 12  Functioning Day             6000 non-null   int64  
 13  year                        6000 

In [35]:
data.isnull().sum() / data.shape[0] * 100

Rented Bike Count             0.0
Hour                          0.0
Temperature(Â°C)              0.0
Humidity(%)                   0.0
Wind speed (m/s)              0.0
Visibility (10m)              0.0
Dew point temperature(Â°C)    0.0
Solar Radiation (MJ/m2)       0.0
Rainfall(mm)                  0.0
Snowfall (cm)                 0.0
Seasons                       0.0
Holiday                       0.0
Functioning Day               0.0
year                          0.0
month                         0.0
day                           0.0
dtype: float64

In [36]:
data.corr()

,Humidity(%),Wind speed (m/s),Visibility (10m),Dew point temperature(Â°C),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm),Holiday,Functioning Day,year,month,day
Humidity(%),1.000000,-0.349406,-0.610871,0.552237,-0.430706,0.233635,0.102155,0.036652,-0.010538,0.023629,0.065512,-0.000466
Wind speed (m/s),-0.349406,1.000000,0.212575,-0.200127,0.331500,-0.047471,-0.015315,-0.049728,-0.068584,0.027586,-0.025679,-0.062063
Visibility (10m),-0.610871,0.212575,1.000000,-0.303082,0.146661,-0.155402,-0.093264,0.021224,0.021274,0.018600,0.026505,-0.179062
Dew point temperature(Â°C),0.552237,-0.200127,-0.303082,1.000000,0.129517,0.127272,-0.163506,0.102185,-0.021761,0.332153,0.033132,0.096683
Solar Radiation (MJ/m2),-0.430706,0.331500,0.146661,0.129517,1.000000,-0.072684,-0.089256,0.026996,-0.049873,0.158746,-0.018207,0.021394
Rainfall(mm),0.233635,-0.047471,-0.155402,0.127272,-0.072684,1.000000,-0.006276,0.005539,0.011703,0.032655,-0.020008,0.003341
Snowfall (cm),0.102155,-0.015315,-0.093264,-0.163506,-0.089256,-0.006276,1.000000,0.006555,0.018375,-0.243541,0.076229,0.114037
Holiday,0.036652,-0.049728,0.021224,0.102185,0.026996,0.005539,0.006555,1.000000,-0.020165,0.142619,-0.005472,-0.016155
Functioning Day,-0.010538,-0.068584,0.021274,-0.021761,-0.049873,0.011703,0.018375,-0.020165,1.000000,-0.033787,-0.130143,0.100204
year,0.023629,0.027586,0.018600,0.332153,0.158746,0.032655,-0.243541,0.142619,-0.033787,1.000000,-0.475476,-0.129710


In [37]:
corr = data.corr()
corr.style.background_gradient(cmap='coolwarm').set_precision(2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: this method is deprecated in favour of `Styler.format(precision=..)`
  


,Humidity(%),Wind speed (m/s),Visibility (10m),Dew point temperature(Â°C),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm),Holiday,Functioning Day,year,month,day
Humidity(%),1.00,-0.35,-0.61,0.55,-0.43,0.23,0.10,0.04,-0.01,0.02,0.07,-0.00
Wind speed (m/s),-0.35,1.00,0.21,-0.20,0.33,-0.05,-0.02,-0.05,-0.07,0.03,-0.03,-0.06
Visibility (10m),-0.61,0.21,1.00,-0.30,0.15,-0.16,-0.09,0.02,0.02,0.02,0.03,-0.18
Dew point temperature(Â°C),0.55,-0.20,-0.30,1.00,0.13,0.13,-0.16,0.10,-0.02,0.33,0.03,0.10
Solar Radiation (MJ/m2),-0.43,0.33,0.15,0.13,1.00,-0.07,-0.09,0.03,-0.05,0.16,-0.02,0.02
Rainfall(mm),0.23,-0.05,-0.16,0.13,-0.07,1.00,-0.01,0.01,0.01,0.03,-0.02,0.00
Snowfall (cm),0.10,-0.02,-0.09,-0.16,-0.09,-0.01,1.00,0.01,0.02,-0.24,0.08,0.11
Holiday,0.04,-0.05,0.02,0.10,0.03,0.01,0.01,1.00,-0.02,0.14,-0.01,-0.02
Functioning Day,-0.01,-0.07,0.02,-0.02,-0.05,0.01,0.02,-0.02,1.00,-0.03,-0.13,0.10
year,0.02,0.03,0.02,0.33,0.16,0.03,-0.24,0.14,-0.03,1.00,-0.48,-0.13


In [38]:
Y_train=data['Rented Bike Count'].values

In [39]:
x=[]
for element in data['Seasons']:
    if element=='Summer':
      x.append(0)
    elif element=='Winter':
      x.append(1)
    elif element=='Spring':
      x.append(2)
    else:
      x.append(3)
data['Seasons']=x  

In [40]:
col=['Hour','Temperature(Â°C)','Humidity(%)','Wind speed (m/s)','Visibility (10m)','Dew point temperature(Â°C)','Solar Radiation (MJ/m2)','Rainfall(mm)','Snowfall (cm)','year','month','day'  ]

In [41]:
x_st=data[col]
object= preprocessing.StandardScaler()
s = object.fit_transform(x_st)
data[col] = s

In [42]:
x_train=data.drop(['Rented Bike Count'],axis=1)


In [43]:
#test
filename="/content/test.csv"
testData = pd.read_csv(filename, encoding= 'unicode_escape', parse_dates=[0,1,2,3])
testData=testData.drop(['ID'],axis=1)

In [44]:
testData['Holiday']=l.fit_transform(testData['Holiday'])
testData['Functioning Day']=l.fit_transform(testData['Functioning Day'])
TestDate=pd.DataFrame({"year": testData['Date'].dt.year,"month": testData['Date'].dt.month,"day": testData['Date'].dt.day})
testData['year']=TestDate["year"]
testData['month']=TestDate["month"]
testData['day']=TestDate["day"]

In [45]:
#from pandas.core.arrays.numeric import T
t=[]

for element in testData['Seasons']:
    if element=='Summer':
      t.append(0)
    elif element=='Winter':
      t.append(1)
    elif element=='Spring':
      t.append(2)
    else:
      t.append(3)
testData['Seasons']=t 
testData.head()

,Date,Hour,Temperature(Â°C),Humidity(%),Wind speed (m/s),Visibility (10m),Dew point temperature(Â°C),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm),Seasons,Holiday,Functioning Day,year,month,day
0,2018-08-08,0,29.1,77,2.1,1465,24.6,0.0,0.0,0.0,0,1,1,2018,8,8
1,2018-08-08,1,28.5,80,1.5,1306,24.7,0.0,0.0,0.0,0,1,1,2018,8,8
2,2018-08-08,2,28.3,84,2.1,1099,25.3,0.0,0.0,0.0,0,1,1,2018,8,8
3,2018-08-08,3,28.3,84,0.5,1006,25.3,0.0,0.0,0.0,0,1,1,2018,8,8
4,2018-08-08,4,28,85,1.0,1079,25.2,0.0,0.0,0.0,0,1,1,2018,8,8


In [46]:
coltest=['Hour','Temperature(Â°C)','Humidity(%)','Wind speed (m/s)','Visibility (10m)','Dew point temperature(Â°C)','Solar Radiation (MJ/m2)','Rainfall(mm)','Snowfall (cm)' ,'year','month','day' ]

In [47]:
testData=testData.drop(['Date'],axis=1)
x_st=testData[coltest]
s2 = object.transform(x_st)
testData[coltest] = s2


In [48]:
lgb_train = lgb.Dataset(x_train, Y_train)

In [49]:
params={}

In [50]:
gbm = lgb.train(params,lgb_train)

In [51]:
y_pre = gbm.predict(testData, num_iteration=gbm.best_iteration)

In [52]:
Results=pd.DataFrame(y_pre, columns = ['Rented Bike Count'])
Results.to_csv('Resultslgbnodstand.csv') 

In [53]:
x_train.head()

,Hour,Temperature(Â°C),Humidity(%),Wind speed (m/s),Visibility (10m),Dew point temperature(Â°C),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm),Seasons,Holiday,Functioning Day,year,month,day
0,-1.661325,-1.268334,-0.960266,0.359711,1.025094,-1.400992,-0.651824,-0.130317,-0.204619,1,1,1,-2.657916,-1.35134,-0.31207
1,-1.516862,-1.291827,-0.912593,-0.951849,1.025094,-1.400992,-0.651824,-0.130317,-0.204619,1,1,1,-2.657916,-1.35134,-0.31207
2,-1.372399,-1.330982,-0.864919,-0.764483,1.025094,-1.408134,-0.651824,-0.130317,-0.204619,1,1,1,-2.657916,-1.35134,-0.31207
3,-1.227936,-1.346644,-0.817246,-0.858166,1.025094,-1.400992,-0.651824,-0.130317,-0.204619,1,1,1,-2.657916,-1.35134,-0.31207
4,-1.083473,-1.330982,-1.007939,0.453394,1.025094,-1.472408,-0.651824,-0.130317,-0.204619,1,1,1,-2.657916,-1.35134,-0.31207


In [54]:
testData.head()

,Hour,Temperature(Â°C),Humidity(%),Wind speed (m/s),Visibility (10m),Dew point temperature(Â°C),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm),Seasons,Holiday,Functioning Day,year,month,day
0,-1.661325,1.417689,0.946663,0.266028,0.164191,1.612757,-0.651824,-0.130317,-0.204619,0,1,1,0.376235,0.712225,-0.740737
1,-1.516862,1.370703,1.089683,-0.296069,-0.091666,1.619898,-0.651824,-0.130317,-0.204619,0,1,1,0.376235,0.712225,-0.740737
2,-1.372399,1.355041,1.280376,0.266028,-0.424763,1.662748,-0.651824,-0.130317,-0.204619,0,1,1,0.376235,0.712225,-0.740737
3,-1.227936,1.355041,1.280376,-1.232897,-0.574416,1.662748,-0.651824,-0.130317,-0.204619,0,1,1,0.376235,0.712225,-0.740737
4,-1.083473,1.331548,1.328049,-0.764483,-0.456947,1.655606,-0.651824,-0.130317,-0.204619,0,1,1,0.376235,0.712225,-0.740737


In [55]:
Results.head()

,Rented Bike Count
0,920.379402
1,731.257131
2,556.176545
3,296.227174
4,241.708925


In [56]:
testData.shape
 

(2760, 15)